In [1]:
pip install torch torchvision albumentations pyyaml opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 143.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 254.8 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 29.9 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 10.7 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 7.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 4.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12
    Uninstalling sympy-1.12:
      Successfully uninstalled sympy-1.12
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages th

In [5]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 482.9 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.1/871.1 kB 4.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 12.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import torch
from torch.utils.data import Dataset
import cv2
import numpy as np
import albumentations as A

class LicensePlateDataset(Dataset):
    def __init__(self, image_dir, annotation_dir, transform=None):
        self.image_dir = image_dir
        self.annotation_dir = annotation_dir
        self.transform = transform
        self.image_files = os.listdir(image_dir)
        self.annotation_files = [f.replace('.jpg', '.txt') for f in self.image_files]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        annotation_path = os.path.join(self.annotation_dir, self.annotation_files[idx])
        
        # Load image
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Load annotations
        boxes = []
        with open(annotation_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                x_center, y_center, width, height = map(float, parts[1:])
                boxes.append([x_center, y_center, width, height])
        
        boxes = np.array(boxes, dtype=np.float32)

        if self.transform:
            transformed = self.transform(image=image, bboxes=boxes)
            image = transformed['image']
            boxes = transformed['bboxes']

        return image, boxes

/Users/mehdiarat/anaconda3/envs/myev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from sklearn.model_selection import KFold
from ultralytics import YOLOv5
from torch.utils.data import DataLoader, Subset

# Dizin yollarını tanımlayın
image_dir = '/Users/mehdiarat/Downloads/archive (1)/images'
annotation_dir = '/Users/mehdiarat/Downloads/archive (1)/annotations'

# K-Fold ayarları
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True)

# YOLO modelini yükleyin
model = YOLO('yolov10n.yaml')  # 'yolov10n.yaml' dosyasını kendi veri setinize göre özelleştirin

# Verilerinizi K-Fold için hazırlayın
dataset = LicensePlateDataset(image_dir=image_dir, annotation_dir=annotation_dir)

# K-Fold işlemi için döngü
for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset)):
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Eğitim ve doğrulama verisetleri için alt küme oluşturun
    train_subsampler = Subset(dataset, train_ids)
    val_subsampler = Subset(dataset, val_ids)

    # DataLoader yapılandırması
    train_loader = DataLoader(train_subsampler, batch_size=4, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_subsampler, batch_size=4, shuffle=False, num_workers=4)
    
    # Eğitim işlemini başlatın
    model.train(data=train_loader, epochs=500, batch=256, imgsz=640)

    # Doğrulama işlemini başlatın
    model.val(data=val_loader, batch=256)

    print(f'Finished fold {fold}')
    print('--------------------------------')

ImportError: cannot import name 'YOLOv5' from 'ultralytics' (/Users/mehdiarat/anaconda3/envs/myev/lib/python3.10/site-packages/ultralytics/__init__.py)